<a href="https://colab.research.google.com/github/jhno-code/2020-2/blob/main/Intro_openMP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
code="""
#include <omp.h>
#include <stdlib.h>
#include <stdio.h>

int main(int argc, char* argv[])
{
  int hilo_ID = 0, num_hilos = 1;
  printf("Region Serializada\\n");

  #pragma omp parallel private(hilo_ID, num_hilos)
  {
    num_hilos = omp_get_num_threads();
    hilo_ID = omp_get_thread_num();
    printf("Hola desde el hilo %d de %d \\n", hilo_ID, num_hilos);
  }
}"""

#creando un archivo para guardar el código
text_file = open("codeA.c", "w")

#Guardar el code en el text_file
text_file.write(code)

#cerrar el archivo text_file
text_file.close()

#definir cuantos hilos se van a usar
%env OMP_NUM_THREADS=4

#ejecutar nuestro código paralelo llamado codeA
!gcc -o Ejecutar -fopenmp codeA.c
!./Ejecutar

env: OMP_NUM_THREADS=4
Region Serializada
Hola desde el hilo 3 de 4 
Hola desde el hilo 1 de 4 
Hola desde el hilo 2 de 4 
Hola desde el hilo 0 de 4 


In [ ]:
code="""
#include <omp.h>
#include <stdlib.h>
#include <stdio.h>


int main(int argc, char* argv[])
{
  int x=1234;
  int iam,np,i;

  #pragma omp parallel private(iam, np, i, x)
  { 
    iam = omp_get_thread_num();
    printf("Soy el thread %d, antes de actualizar, con x=%d\\n",iam,x);
    x=iam*1111;
    printf("\t\tSoy el thread %d, despues de actualizar, con x=%d\\n",iam,x);
  }
  
printf("\\n Despues de pragma parallel x=%d \\n",x); //x=1234
}
"""

text_file = open("codeB.c", "w")
text_file.write(code)
text_file.close()
%env OMP_NUM_THREADS=4
!gcc -o Ejecutar -fopenmp codeB.c
!./Ejecutar

env: OMP_NUM_THREADS=4
Soy el thread 0, antes de actualizar, con x=0
		Soy el thread 0, despues de actualizar, con x=0
Soy el thread 1, antes de actualizar, con x=0
		Soy el thread 1, despues de actualizar, con x=1111
Soy el thread 2, antes de actualizar, con x=0
		Soy el thread 2, despues de actualizar, con x=2222
Soy el thread 3, antes de actualizar, con x=0
		Soy el thread 3, despues de actualizar, con x=3333

 Despues de pragma parallel x=1234 


In [9]:
code="""
#include <omp.h>
#include <stdlib.h>
#include <stdio.h>


int main(int argc, char* argv[])
{
  int TID, i, vlen, n, indx;
  indx = 4;             // desplazamiento
  n = 2;                // nro de datos por hilo
  //vlen = indx + n * 4;  // 4 = nro de hilos
  vlen = indx + n * 4 + 5;

  int a[vlen];
  for(int i=0; i < vlen; i++) a[i] = -i-1;  // a = [-1, -2, -3, ...]

  #pragma omp parallel private(i,TID) firstprivate(indx) shared(n,a)
  {
      TID = omp_get_thread_num();       // obtener numero de hilo
      indx += n*TID;                    // desplazamiento para cada hilo
      for(i=indx; i<indx+n; i++)        // Cambiar valores de "a"
          a[i] = TID + 1;
  } /*-- End of parallel region --*/

  printf("Despues de la región paralela:\\n");
  for (i=0; i<vlen; i++)
    printf("a[%d] = %d\\n",i,a[i]);     // imprimir elementos de "a"
  return 0;
}
"""

text_file = open("codeB.c", "w")
text_file.write(code)
text_file.close()
%env OMP_NUM_THREADS=4
!gcc -o Ejecutar -fopenmp codeB.c
!./Ejecutar

env: OMP_NUM_THREADS=4
Despues de la región paralela:
a[0] = -1
a[1] = -2
a[2] = -3
a[3] = -4
a[4] = 1
a[5] = 1
a[6] = 2
a[7] = 2
a[8] = 3
a[9] = 3
a[10] = 4
a[11] = 4
a[12] = -13
a[13] = -14
a[14] = -15
a[15] = -16
a[16] = -17
